In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from random import randrange
from skimage import exposure
from skimage.exposure import match_histograms

#### Histogram matching of templates

In [ ]:
template = cv2.imread("E:/C/project_final/t.jpg")
sun = cv2.imread("E:/C/project_final/IMG_2.jpg") #load image of a template in natural light
bulb = cv2.imread("E:/C/project_final/IMG_1.jpg") #load image of a template in artificial light
sun_matched = match_histograms(sun, template, channel_axis=-1) #histogram matching of template in natural light with digital template
bulb_matched = match_histograms(bulb, template, channel_axis=-1) #histogram matching of template in artificial light with digital template

#show templates: digital, artificial light, matched artificial light, natural light, matched natural light
fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(25,15))

ax[0].imshow(cv2.cvtColor(template, cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(bulb, cv2.COLOR_BGR2RGB))
ax[2].imshow(cv2.cvtColor(bulb_matched, cv2.COLOR_BGR2RGB))
ax[3].imshow(cv2.cvtColor(sun, cv2.COLOR_BGR2RGB))
ax[4].imshow(cv2.cvtColor(sun_matched, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
#show R,G,B histograms for templates: digital, artificial light, matched artificial light, natural light, matched natural light

fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25,15))

for i, img in enumerate((template, sun, sun_matched, bulb, bulb_matched)):
    for c, c_color in enumerate(('red', 'green', 'blue')):
        img_hist, bins = exposure.histogram(img[..., c], source_range='dtype')
        axes[c, i].plot(bins, img_hist / img_hist.max())
        img_cdf, bins = exposure.cumulative_distribution(img[..., c])
        axes[c, i].plot(bins, img_cdf)
        axes[c, 0].set_ylabel(c_color, fontsize=20)

#axes[0, 0].set_title('template')
#axes[0, 1].set_title('sun template')
#axes[0, 2].set_title('sun matched template')
#axes[0, 3].set_title('bulb template')
#axes[0, 4].set_title('bulb matched template')
plt.legend(["histogram", "cumulative histogram"], fontsize=16)

plt.tight_layout()
plt.show()

#### Data preparation

In [ ]:
CATEGORIES = ["0","20","50","80","100","150","200","300","400"] #additives of vitamin C

In [ ]:
# histogram matched artificial light data
for cat in CATEGORIES:
    
    # assign directory
    directory = 'E:/C/project_final/sun/'+str(cat)

    iter=-1
    # iterate over files in directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            iter=iter+1

            #read image and select ROI
            image=cv2.imread(f)
            img_col = image[800:1200, 2100:2500]

            colors = ("blue", "green", "red")
            d=-1
            imres = img_col.copy()
            #repeat adapated histogram matching for R, G, B channels
            for channel_id, color in enumerate(colors):
                d=d+1
                histogram, bin_edges = np.histogram(
                    img_col[:, :, channel_id], bins=256, range=(0, 256)
                )
                histogram_1, bin_edges = np.histogram(
                    sun_matched[:, :, channel_id], bins=256, range=(0, 256)
                )
                
                src_cdf = np.cumsum(histogram) #cumulative distribution function
                src_cdf = (255 * src_cdf / src_cdf[-1]).astype(np.uint8) #normalize

                #adapt histogram of template to include only relevant range
                pix = 0
                ref_histogram=histogram_1
                for i in range(256):
                    if histogram[i]>=0.01*np.max(histogram):
                        ref_histogram[i]=histogram_1[i]
                    else:
                        ref_histogram[i]=0
                        pix = pix+histogram_1[i]

                s = np.random.randint(0,256,pix)
                for i in range(256):
                    x=[j for j in s if j==i]
                    ref_histogram[i]=ref_histogram[i]+len(x)
                
                dst_cdf = np.cumsum(ref_histogram) #cumulative distribution function
                dst_cdf = (255 * dst_cdf / dst_cdf[-1]).astype(np.uint8) #normalize

                im2 = np.interp(imres[:,:,d].flatten(),bin_edges[:-1],src_cdf)

                #map values
                im3 = np.interp(im2,dst_cdf, bin_edges[:-1])

                imres[:,:,d] = im3.reshape((img_col.shape[0],img_col.shape[1])) #reshape data

            cv2.imwrite('C:/Users/User/Desktop/Rivendell/sun/const/'+str(cat)+'/'+str(iter)+'.jpg',imres)

In [ ]:
# histogram matched artificial light data
for cat in CATEGORIES:
    
    # assign directory
    directory = 'E:/C/project_final/bulb/'+str(cat)

    iter=-1
    # iterate over files in directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            iter=iter+1

            #read image and select ROI
            image=cv2.imread(f)
            img_col = image[800:1200, 2100:2500]

            colors = ("blue", "green", "red")
            d=-1
            imres = img_col.copy()
            #repeat adapated histogram matching for R, G, B channels
            for channel_id, color in enumerate(colors):
                d=d+1
                histogram, bin_edges = np.histogram(
                    img_col[:, :, channel_id], bins=256, range=(0, 256)
                )
                histogram_1, bin_edges = np.histogram(
                    bulb_matched[:, :, channel_id], bins=256, range=(0, 256)
                )
                
                src_cdf = np.cumsum(histogram) #cumulative distribution function
                src_cdf = (255 * src_cdf / src_cdf[-1]).astype(np.uint8) #normalize

                #adapt histogram of template to include only relevant range
                pix = 0
                ref_histogram=histogram_1
                for i in range(256):
                    if histogram[i]>=0.01*np.max(histogram):
                        ref_histogram[i]=histogram_1[i]
                    else:
                        ref_histogram[i]=0
                        pix = pix+histogram_1[i]

                s = np.random.randint(0,256,pix)
                for i in range(256):
                    x=[j for j in s if j==i]
                    ref_histogram[i]=ref_histogram[i]+len(x)
                
                dst_cdf = np.cumsum(ref_histogram) #cumulative distribution function
                dst_cdf = (255 * dst_cdf / dst_cdf[-1]).astype(np.uint8) #normalize

                im2 = np.interp(imres[:,:,d].flatten(),bin_edges[:-1],src_cdf)

                #map values
                im3 = np.interp(im2,dst_cdf, bin_edges[:-1])

                imres[:,:,d] = im3.reshape((img_col.shape[0],img_col.shape[1])) #reshape data

            cv2.imwrite('C:/Users/User/Desktop/Rivendell/bulb/const/'+str(cat)+'/'+str(iter)+'.jpg',imres)

In [ ]:
# natural light data
for cat in CATEGORIES:
    
    # assign directory
    directory = 'E:/C/project_final/sun/'+str(cat)

    iter=-1
    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):

            iter=iter+1

            image=cv2.imread(f)
            imres = image[800:1200, 2100:2500]

            cv2.imwrite('C:/Users/User/Desktop/Rivendell/sun/org/'+str(cat)+'/'+str(iter)+'.jpg',imres)

In [ ]:
# natural light data
for cat in CATEGORIES:
    
    # assign directory
    directory = 'E:/C/project_final/bulb/'+str(cat)

    iter=-1
    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):

            iter=iter+1

            image=cv2.imread(f)
            imres = image[800:1200, 2100:2500]

            cv2.imwrite('C:/Users/User/Desktop/Rivendell/bulb/org/'+str(cat)+'/'+str(iter)+'.jpg',imres)